# Loading the data and pre process

In [1]:
import pandas as pd

# Load the dataset (Ensure the correct path)
df = pd.read_csv("Raw_labeled_with_gemini.csv")

# Display basic info
print(df.info())
print(df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 779 entries, 0 to 778
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            779 non-null    object
 1   image            506 non-null    object
 2   published        779 non-null    object
 3   description      775 non-null    object
 4   category         779 non-null    object
 5   author           762 non-null    object
 6   image_relation   779 non-null    object
 7   fake_news_label  779 non-null    object
dtypes: object(8)
memory usage: 48.8+ KB
None
                                               title  \
0  Social Security head replaced after clash with...   
1  Richard Linklater and Ethan Hawke Tease New Mo...   
2  Northpoint Commercial Finance Partners with Ya...   
3  Institutions Can Earn Bitcoin Yield With lstBT...   
4  Your 10-Step Guide to Buying a Home, From Star...   

                                               image  \
0  http

In [2]:

# Standardize column names
df.columns = df.columns.str.strip().str.lower()

# Fill missing values
df["image"].fillna("Unknown", inplace=True)
df["author"].fillna("Unknown", inplace=True)
df["description"].fillna("No description available", inplace=True)

# Fix category format (Convert ['category'] → category)
df["category"] = df["category"].str.replace(r"[\[\]']", "", regex=True)

# Convert 'published' column to datetime format
df["published"] = pd.to_datetime(df["published"], errors="coerce")

# Remove duplicates
df.drop_duplicates(inplace=True)

# Save the cleaned dataset
df.to_csv("cleaned_news_data.csv", index=False)

print("✅ Preprocessing complete! Cleaned data saved as 'cleaned_news_data.csv'")


✅ Preprocessing complete! Cleaned data saved as 'cleaned_news_data.csv'


<ipython-input-2-6be059917ab1>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["image"].fillna("Unknown", inplace=True)
<ipython-input-2-6be059917ab1>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.m

In [ ]:
import os
import re
import pandas as pd
import torch
import torchvision.transforms as transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

# Load dataset
file_path = r'C:\Users\rajar\OneDrive\Desktop\New folder\Fake-News\All_Data\Cleaned_news_final1.csv'
df = pd.read_csv(file_path)

# Manual stopwords list
manual_stopwords = set([
    "a", "an", "and", "are", "as", "at", "be", "but", "by", "for", "if", "in", "into", "is", 
    "it", "no", "not", "of", "on", "or", "such", "that", "the", "their", "then", "there", 
    "these", "they", "this", "to", "was", "will", "with", "we", "you", "your"
])

# Function to preprocess text
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = ' '.join([word for word in text.split() if word not in manual_stopwords])
    return text

# Apply text preprocessing
df['cleaned_title'] = df['title'].apply(preprocess_text)
df['cleaned_description'] = df['description'].apply(preprocess_text)

# Tokenize text using BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
df['tokenized_text'] = df['cleaned_description'].apply(lambda x: tokenizer(x, padding='max_length', truncation=True, max_length=512, return_tensors='pt'))

# Define image transformation pipeline
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define image directory path
IMAGE_DIR = r"C:\Users\rajar\Projects\Fake-News\Fake-News\Images" 

# Function to load and preprocess images
def load_and_preprocess_image(image_filename):
    image_path = os.path.join(IMAGE_DIR, image_filename)  # Construct full path
    if os.path.exists(image_path):
        image = Image.open(image_path).convert("RGB")
        return image_transform(image)
    return None

# Apply image preprocessing
df['processed_image'] = df['image_location'].apply(lambda x: load_and_preprocess_image(x) if isinstance(x, str) else None)

# Split data into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

print("Data processing complete!")
